In [1]:
import numpy as np
import pandas as pd
from libsvm.svmutil import * #High-level usage
import scipy
import time
import gc

In [2]:
y_train, X_train = svm_read_problem('./satimage.scale', return_scipy = True)
y_test, X_test = svm_read_problem('./satimage.scale.t', return_scipy = True)

## Problem 11
- primal: original X
- dual: linear kernel
- params
    - C: 10
    - classification: 5 v.s. not 5

In [4]:
y_train_transformed = [-1 if i == 5 else 1 for i in y_train]
y_test_transformed = [-1 if i == 5 else 1 for i in y_test]

In [7]:
prob  = svm_problem(y_train_transformed, X_train)
param = svm_parameter('-s 0 -t 0 -c 10 -e 0.000001')
m = svm_train(prob, param)
p_label, p_acc, p_val = svm_predict(y_test_transformed, X_test, m)
ACC, MSE, SCC = evaluations(y_test_transformed, p_label)

Accuracy = 91.85% (1837/2000) (classification)


In [8]:
coefs = m.get_sv_coef()
values = m.get_SV()

In [9]:
coefs = np.asarray(coefs)
SV_values = np.zeros(
    (len(values), X_train.shape[1])
)
for coef_idx, coef in enumerate(coefs):
    for value_idx, value in values[coef_idx].items():
        SV_values[coef_idx][value_idx-1] = value

In [10]:
w = np.sum(coefs * SV_values, axis = 0)
norm_w = np.linalg.norm(w)
print(f"W norm is {norm_w}")

W norm is 4.644324255398067


## Problem 12 / 13
- polynomial kernel
- degree = 3
- C = 10

In [21]:
y_train, X_train = svm_read_problem('./satimage.scale', return_scipy = True)
y_test, X_test = svm_read_problem('./satimage.scale.t', return_scipy = True)

class_list = [2, 3, 4, 5, 6]
max_num_SV = 0
max_num_SV_class = 0
for label in class_list:
    print(f"When specified class label is {label}")
    y_train_transformed = [-1 if i == label else 1 for i in y_train]
    prob  = svm_problem(y_train_transformed, X_train)
    param = svm_parameter('-s 0 -t 1 -d 3 -r 1 -g 1 -c 10')
    m = svm_train(prob, param)
    p_label, p_acc, p_val = svm_predict(y_train_transformed, X_train, m)
    #ACC, MSE, SCC = evaluations(y_test_transformed, p_label)

    #compute number of SVs
    num_SV = len(m.get_SV())
    print(f"Number of SV is {num_SV}")
    if num_SV > max_num_SV:
        max_num_SV_class = label
        max_num_SV = num_SV

print(f"The class with maximum number of SV is {max_num_SV_class}, has {max_num_SV} SVs")

When specified class label is 2
Accuracy = 100% (4435/4435) (classification)
Number of SV is 93
When specified class label is 3
Accuracy = 99.7294% (4423/4435) (classification)
Number of SV is 385
When specified class label is 4
Accuracy = 99.1657% (4398/4435) (classification)
Number of SV is 660
When specified class label is 5
Accuracy = 100% (4435/4435) (classification)
Number of SV is 281
When specified class label is 6
Accuracy = 99.7069% (4422/4435) (classification)
Number of SV is 607
The class with maximum number of SV is 4, has 660 SVs


## Problem 14
- Gaussian kernel
- 1 v.s. not 1
- gamma = 10
- test E_out with C = [0.01, 0.1, 1, 10, 100]

In [26]:
y_train_transformed = [-1 if i == 1 else 1 for i in y_train]
y_test_transformed = [-1 if i == 1 else 1 for i in y_test]
Cs = [0.01, 0.1, 1, 10, 100]

best_c = 0
best_acc = 0

for c in Cs:
    print(f"When C = {c}")
    prob  = svm_problem(y_train_transformed, X_train)
    param = svm_parameter(f'-s 0 -t 2 -g 10 -c {c} -e 0.000001')
    m = svm_train(prob, param)
    p_label, p_acc, p_val = svm_predict(y_test_transformed, X_test, m)
    ACC, MSE, SCC = evaluations(y_test_transformed, p_label)
    if ACC > best_acc:
        best_c = c
        best_acc = ACC
    if ACC == best_acc and c < best_c:
        best_c = c
        best_acc = ACC

print(f"Best result: C = {best_c}, accuracy = {best_acc}%")

When C = 0.01
Accuracy = 76.95% (1539/2000) (classification)
When C = 0.1
Accuracy = 79.25% (1585/2000) (classification)
When C = 1
Accuracy = 89.05% (1781/2000) (classification)
When C = 10
Accuracy = 89.95% (1799/2000) (classification)
When C = 100
Accuracy = 89.95% (1799/2000) (classification)
Best result: C = 10, accuracy = 89.95%


# Problem15
- C = 0.1
- test E_out with Gamma = [0.1, 1, 10, 100, 1000]

In [28]:
y_train_transformed = [-1 if i == 1 else 1 for i in y_train]
y_test_transformed = [-1 if i == 1 else 1 for i in y_test]
Gammas = [0.1, 1, 10, 100, 1000]

best_g = 0
best_acc = 0

for gamma in Gammas:
    print(f"When Gamma = {gamma}")
    prob  = svm_problem(y_train_transformed, X_train)
    param = svm_parameter(f'-s 0 -t 2 -g {gamma} -c 0.1 -e 0.000001')
    m = svm_train(prob, param)
    p_label, p_acc, p_val = svm_predict(y_test_transformed, X_test, m)
    ACC, MSE, SCC = evaluations(y_test_transformed, p_label)
    if ACC > best_acc:
        best_g = gamma
        best_acc = ACC
    if ACC == best_acc and gamma < best_g:
        best_g = gamma
        best_acc = ACC

print(f"Best result: gamma = {best_g}, accuracy = {best_acc}%")

When Gamma = 0.1
Accuracy = 98.75% (1975/2000) (classification)
When Gamma = 1
Accuracy = 98.8% (1976/2000) (classification)
When Gamma = 10
Accuracy = 79.25% (1585/2000) (classification)
When Gamma = 100
Accuracy = 76.95% (1539/2000) (classification)
When Gamma = 1000
Accuracy = 76.95% (1539/2000) (classification)
Best result: gamma = 1, accuracy = 98.8%


In [ ]:
# When Gamma = 0.1
# Accuracy = 98.75% (1975/2000) (classification)
# When Gamma = 1
# Accuracy = 98.8% (1976/2000) (classification)
# When Gamma = 10
# Accuracy = 79.25% (1585/2000) (classification)
# When Gamma = 100
# Accuracy = 76.95% (1539/2000) (classification)
# When Gamma = 1000
# Accuracy = 76.95% (1539/2000) (classification)
# Best result: gamma = 1, accuracy = 98.8%

## Problem 16
- validation with 200 examples
- C = 0.1
- repeat procedures 1000 times

In [17]:
num_data = X_train.shape[0]
total_idx = np.arange(0, num_data)

Gammas = [0.1, 1, 10, 100, 1000]
gamma_best_count = {
    0.1: 0, 1:0, 10:0,
    100:0, 1000:0
}

NUM_EXPERIMENT = 1000

start = time.time()
for i in range(NUM_EXPERIMENT):
    print(f"Experiment {i+1} running...", end = " ")

    #train-valid split
    valid_idx = np.random.choice(total_idx, 200, replace = False)
    subtrain_idx = np.delete(total_idx, valid_idx)

    X_subtrain = X_train[subtrain_idx, :]
    y_subtrain = y_train[subtrain_idx]
    X_valid = X_train[valid_idx, :]
    y_valid = y_train[valid_idx]

    y_subtrain_transformed = [-1 if i == 1 else 1 for i in y_subtrain]
    y_valid_transformed = [-1 if i == 1 else 1 for i in y_valid]

    best_g = 0
    best_acc = 0
    for gamma in Gammas:
        #print(f"When Gamma = {gamma}")
        prob  = svm_problem(y_subtrain_transformed, X_subtrain)
        param = svm_parameter(f'-s 0 -t 2 -c 0.1 -g {gamma}')
        m = svm_train(prob, param)

        p_label, p_acc, p_val = svm_predict(y_valid_transformed, X_valid, m, "-q")
        ACC, MSE, SCC = evaluations(y_valid_transformed, p_label)
        if ACC > best_acc:
            best_g = gamma
            best_acc = ACC

    #print(best_g)
    gamma_best_count[best_g] += 1
    end = time.time()
    t_dur = (end - start) / 60
    print(f"{round(t_dur, 3)} min elapsed.")
    gc.collect()

Experiment 1 running... 0.019 min elapsed.
Experiment 2 running... 0.038 min elapsed.
Experiment 3 running... 0.057 min elapsed.
Experiment 4 running... 0.076 min elapsed.
Experiment 5 running... 0.095 min elapsed.
Experiment 6 running... 0.114 min elapsed.
Experiment 7 running... 0.134 min elapsed.
Experiment 8 running... 0.154 min elapsed.
Experiment 9 running... 0.173 min elapsed.
Experiment 10 running... 0.192 min elapsed.
Experiment 11 running... 0.211 min elapsed.
Experiment 12 running... 0.23 min elapsed.
Experiment 13 running... 0.249 min elapsed.
Experiment 14 running... 0.269 min elapsed.
Experiment 15 running... 0.288 min elapsed.
Experiment 16 running... 0.307 min elapsed.
Experiment 17 running... 0.326 min elapsed.
Experiment 18 running... 0.345 min elapsed.
Experiment 19 running... 0.364 min elapsed.
Experiment 20 running... 0.383 min elapsed.
Experiment 21 running... 0.402 min elapsed.
Experiment 22 running... 0.421 min elapsed.
Experiment 23 running... 0.44 min elapsed.

In [18]:
print(gamma_best_count)

{0.1: 746, 1: 254, 10: 0, 100: 0, 1000: 0}
